In [60]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
import joblib
import matplotlib.pyplot as plt
import matplotlib as mpl
import highlight_text
import matplotlib.font_manager
from io import BytesIO
import base64
import math
from scipy import stats
import json

In [133]:
def create_files(position):
    df = pd.read_csv('./Data/ws_df_sim.csv')

    df_21 = df[df.Year == 2021]
    df_20 = df[df.Year == 2020]
    df_19 = df[df.Year == 2019]

    df_21= df_21.drop([	'tournamentName', 'firstName', 'lastName', 'SecondName', 'FirstLastName',
        'age', 'Full name'] , axis = 1)
    df_20= df_20.drop([	'tournamentName','firstName', 'lastName', 'SecondName', 'FirstLastName',
        'age', 'Full name'] , axis = 1)
    df_19= df_19.drop([	'tournamentName', 'firstName', 'lastName', 'SecondName', 'FirstLastName',
        'age', 'Full name'] , axis = 1)

    df_all = df_21.join(
        df_20.set_index(['playerId']),
        lsuffix="_x",
        rsuffix="_y",
        on=["playerId"],
    ).join(
        df_19.set_index(['playerId']),
        lsuffix="_x",
        rsuffix="_y",
        on=["playerId"],
    )

    df_all = df_all.dropna()  

    df_all = df_all.drop([ 'NewName_y',  'positionText_y',
    'NewName', 'positionText', 'League_y', 'League', 'teamName_y', 'teamName' ], axis = 1)

    df_cluster = df_all.copy()
    df_cluster.to_csv('./Data/ws_sim_player.csv')
    df_21.to_csv('./Data/df_21.csv')

    df_clust = df_cluster[df_cluster.positionText_x == position]
    df_columns = df_clust[['NewName_x',	'playerId',	'positionText_x', 'teamName_x', 'League_x']]
    df_columns = df_columns.reset_index()
    #index = df_columns[df_columns.NewName_x==name].index

    df_clust = df_clust.drop(['NewName_x',	'playerId',	'positionText_x', 'Year', 'Year_x', 'Year_y', 'teamName_x', 'League_x'], axis = 1)

    df_columns.to_csv('./Data/df_columns_' + position + '.csv')

    num = 5
    pca = PCA(n_components=num)
    principalComponents = pca.fit_transform(df_clust)
    principalDf = pd.DataFrame(data = principalComponents
                , columns = [f'principal_{i}' for i in range(num)])

    principalDf.to_csv('./Data/ws_sim_' + position + '.csv')
    pca_path = './Models/pca_' + position + '.sav'
    joblib.dump(pca, pca_path)

In [136]:
position = 'Forward'
create_files(position)